In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls

adc.json  drive  sample_data


In [1]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [3]:
lidar = cv2.imread("pca/lidar.png", cv2.IMREAD_GRAYSCALE)
lidar = lidar.reshape(1, lidar.shape[0], lidar.shape[1])
print(lidar.shape)

(1, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
lidar_train_data = []
lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            lidar_train_data.append(lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            lidar_test_data.append(lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
lidar_train_data = np.array(lidar_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [8]:
print(lidar_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 1)
(2832,)
(12197,)


In [9]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [10]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [12]:
padded_lidar = np.lib.pad(lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')
print(padded_lidar.shape)

(1, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
lidar_train_patches = []
lidar_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      lidar_train_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      lidar_test_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [15]:
lidar_train_patches = np.array(lidar_train_patches)
lidar_test_patches = np.array(lidar_test_patches)
print(lidar_train_patches.shape)
print(lidar_test_patches.shape)

(2832, 41, 41, 1)
(12197, 41, 41, 1)


In [44]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

lidar_model = Sequential()
lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
lidar_model.add(BatchNormalization())
lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
lidar_model.add(BatchNormalization())
lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


lidar_model.add(Flatten())
lidar_model.add(Dense(FC1, activation='relu'))
lidar_model.add(Dropout(0.75))
lidar_model.add(Dense(FC2, activation='relu'))
lidar_model.add(Dropout(0.6))
lidar_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
lidar_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

lidar_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 21, 21, 500)       18500     
_________________________________________________________________
batch_normalization_2 (Batch (None, 21, 21, 500)       2000      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 500)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 5, 5, 100)         1250100   
_________________________________________________________________
batch_normalization_3 (Batch (None, 5, 5, 100)         400       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
__________

In [48]:
history = lidar_model.fit(lidar_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1286 - acc: 0.9626
Epoch 2/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1446 - acc: 0.9555
Epoch 3/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1315 - acc: 0.9601
Epoch 4/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1436 - acc: 0.9562
Epoch 5/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1630 - acc: 0.9499
Epoch 6/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1521 - acc: 0.9573
Epoch 7/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1620 - acc: 0.9495
Epoch 8/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1692 - acc: 0.9530
Epoch 9/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1545 - acc: 0.9523
Epoch 10/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.1574 - acc: 0.9499
Epoch 11/50
2832/2832 [======

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [46]:
test_cls = test_labels - 1
prediction = lidar_model.predict(lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.5034844633926375
Average Accuracy of training samples :  0.5351069735432643
Kappa statistics of training samples :  0.4611027347129306
Each class accuracy of training samples :  [0.38403042 0.37081851 0.57446809 0.78197065 0.52842105 0.55665025
 0.50909091 0.61577181 0.38831615 0.79407407 0.82731707 0.27564506
 0.24725275 0.81533101 0.35744681]
Confusion matrix : [[101   9   0   0  29  16   1   0  39  41   2   0   4   2  19]
 [177 521  44  39  66   1  12  26  80  79  42 140  42   7 129]
 [  7   0 216   0   0   0   0   0   1   0   0   0   0   0 152]
 [ 11   0   0 373   3   0  64  12   6   8   0   0   0   0   0]
 [ 88  42   0   0 251   0   0   0  17  33   3  20  20   0   1]
 [ 17   4   0   7  38 113   0   0   1  14   0   7   2   0   0]
 [ 20  10   0 448  34   4 840 107  94  40  43  10   0   0   0]
 [  9   3   0 106   5   0  34 734   0   8   0 292   0   0   1]
 [ 49 210   0  42 439   7  89   2 678  47  65  61  56   0   1]
 [ 15  16   0   7   1   0

In [0]:
lidar_model.save_weights('Models/lidar_model_weights.h5')